<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Limit_and_offset_concept_in_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing the required module**

In [ ]:
import requests
import uuid
import json

# **Defining the limit & initial offset**

In [ ]:
offset=0
limit=1000

# **Creating the folder where jsons for each iterations will be stored**

In [ ]:
!mkdir /content/extracted_data

# **Pagination Code**

In [ ]:
while True:
  url="http://jsonplaceholder.typicode.com/photos?_start={}&_limit={}".format(offset,limit)
  print("URL for the current iteration : {}".format(url))
  result=requests.get(url);
  data_extracted=result.json();
  if(len(data_extracted)==0):
    break;
  else:
    file_name="/content/extracted_data/"+str(uuid.uuid4())+".json"
    with open(file_name, 'w') as fp:
      json.dump(data_extracted, fp)
    offset=offset+limit

# **Installing Spark**

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession

# **Build the sparkSession**

In [ ]:
spark=SparkSession.builder.appName("Hello").getOrCreate()

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# **Reading all the json files created in the earlier steps**

In [ ]:
df1 = spark.read.option("multiline", True).option("inferSchema", False).json("/content/extracted_data/")

In [ ]:
df1.show()

# **Reading the csv file processed using https://jsonformatter.org/405ecb# from http://jsonplaceholder.typicode.com/photos**

In [ ]:
df2 = spark.read.format('csv').option("header", True).option("inferSchema", False).load("/content/items_processed_total.csv")

In [ ]:
df2.show()

In [ ]:
df1.columns

# **Make the column order same for both dataframes**

In [ ]:
df3=df2.select(['albumId', 'id', 'thumbnailUrl', 'title', 'url'])

In [ ]:
(df1.subtract(df3)).count()

In [ ]:
(df3.subtract(df1)).count()